In [1]:
%gui qt
%matplotlib qt
from xmcd_projection import *
import matplotlib.pyplot as plt
from glob import glob
import os
from skimage.io import imread, imsave

c:\users\lukas\miniconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\lukas\miniconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\users\lukas\miniconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Get file paths

In [ ]:
msh_file = "example_mesh.msh"
mag_file = "mag_data.csv"

## Generate raytracing - skip if generated

In [7]:
# get the mesh
msh = Mesh.from_file(msh_file)

#### Make sure that the projection vector is correct and that the structure is oriented well

In [8]:
# get the projection vector
p = get_projection_vector(90, 16)

# prepare raytracing object
raytr = RayTracing(msh, p)
struct = raytr.struct
struct_projected = raytr.struct_projected

In [12]:
vis = MeshVisualizer(struct, struct_projected)
vis.set_camera(dist=2e5)
vis.show()

## If raytracing file generated - skip if not

In [26]:
# load raytracing if exists
raytr = np.load("raytracing.npy", allow_pickle=True).item()
struct = raytr.struct
struct_projected = raytr.struct_projected

## Save raytracing

In [13]:
raytr.get_piercings()
np.save("raytracing.npy", raytr, allow_pickle=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 5490/5490 [00:06<00:00, 798.10it/s]


## Get the xmcd

#### Get magnetisation, fix vertex shuffling
Note: sometimes if the mesh file has multiple parts, the paraview export and the original mesh coordinates are not in the same order. I add a function to fix that when necessary

In [15]:
magnetisation, mag_points = load_mesh_magnetisation(mag_file)
shuffle_file = "shuffle_indx.npy"
try:
    shuffle_indx = np.load(shuffle_file)
except FileNotFoundError:
    print('File not found. Generating shuffle indx')
    shuffle_indx = msh.get_shuffle_indx(mag_points)
    np.save(shuffle_file, shuffle_indx)
magnetisation = magnetisation[shuffle_indx, :]

File not found. Generating shuffle indx


### Get the colours and XMCD values

In [16]:
xmcd_value = raytr.get_xmcd(magnetisation)
mag_colors = get_struct_face_mag_color(struct, magnetisation)

In [21]:
# deep_reload(xmcd_projection)
# from xmcd_projection import *
azi=90
center_struct = [0, 0, 0]
dist_struct = 2e4
center_peem = [0, -1000, 0]
dist_peem = 2e5

vis = MeshVisualizer(struct, struct_projected, projected_xmcd=xmcd_value, struct_colors=mag_colors)
vis.show(azi=azi, center=center_peem, dist=dist_peem)

#### View different parts of the image separately

#### Both

In [22]:
vis.update_colors(xmcd_value, mag_colors)
vis.view_both(azi=azi, center=center_peem, dist=dist_peem)

#### Projection

In [23]:
vis.view_projection(azi=azi, center=center_peem, dist=dist_peem)

#### Structure

In [24]:
vis.view_struct(azi=azi, center=center_struct, dist=dist_struct)

#### Saving one render

In [25]:
vis.view_both(azi=azi, center=center_peem, dist=dist_peem)
vis.save_render('shadow.png')
vis.view_projection()
blurred = vis.get_blurred_image(desired_background=0.7)
imsave('shadow_blurred.png', (blurred*255).astype(np.uint8), check_contrast=False)

vis.view_struct(azi=azi, center=center_struct, dist=dist_struct)
vis.save_render('structure_view.png')